# Gender Difference: XGB Experiment based on 59 Variables 1513k Rows 14 Waves

In [1]:
%pwd

'D:\\OneDrive - Kyushu University\\ESG09_Article\\Code'

In [2]:
%cd ..

D:\OneDrive - Kyushu University\ESG09_Article


## Import Packagee

In [3]:
import os 
import pandas as pd
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

## Load and Make Datasets

In [4]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [5]:
Df = pd.read_parquet(Df_Filename)

In [6]:
Df.shape

(1911212, 64)

In [7]:
Df.columns

Index(['wave', 'INCOME_2', 'Cantril_ladder', 'Health_disable', 'Relative_have',
       'Living_standard_change', 'Enough_food', 'Enough_shelter',
       'Well_rested', 'Respected', 'Smile', 'Interesting_thing', 'Enjoyment',
       'Physical_pain', 'Worry', 'Sadness', 'Stress', 'Anger',
       'City_satisficied', 'Economic_change', 'Goodtime_job', 'Sat_pubtran',
       'Sat_road', 'Sat_edu', 'Sat_qualityair', 'Sat_qualitywater',
       'Sat_healthcare', 'Sat_affhouse', 'Sat_oppofriend', 'Good_minorities',
       'Good_gayles', 'Good_immigrants', 'Donated', 'Volunteer',
       'Help_stranger', 'Voice_official', 'Local_police', 'Safety_walk',
       'Stolen', 'Assualted', 'Religion_importance', 'Children_respected',
       'Children_learn', 'Women_respected', 'Sat_dealpoor', 'Sat_perserveenv',
       'Freedom_chooselife', 'Conf_military', 'Conf_judicial',
       'Conf_government', 'Conf_financial', 'Conf_honestyelections',
       'Freedom_media', 'Corruption_business', 'Corruption_governm

### Check the Difference between Gender

In [8]:
female_cantrilladder = Df.loc[Df['Gender_female']==1, 'Cantril_ladder'].mean()

In [9]:
female_cantrilladder

5.569313150299246

In [10]:
male_cantrilladder = Df.loc[Df['Gender_female']==0, 'Cantril_ladder'].mean()

In [11]:
male_cantrilladder

5.466124824941722

In [12]:
t_stat, p_value = stats.ttest_ind(Df.loc[Df['Gender_female']==1, 'Cantril_ladder'], Df.loc[Df['Gender_female']==0, 'Cantril_ladder'])

In [13]:
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: 29.49502322170667, P-value: 3.682499213688914e-191


### Shuffle Conversion

In [14]:
Df = Df.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

### Df_male 

In [16]:
Df_male = Df.loc[Df['Gender_female']==0, :].drop(columns=['Gender_female'])

In [17]:
Df_male.shape

(893988, 63)

In [18]:
ym = Df_male['Cantril_ladder']

In [19]:
Xm = Df_male.drop(columns=['Cantril_ladder'])

In [20]:
Xm_train, Xm_test, ym_train, ym_test = train_test_split(Xm, ym, test_size=0.1, random_state=42)

### Df_female 

In [21]:
Df_female = Df.loc[Df['Gender_female']==1, :].drop(columns=['Gender_female'])

In [22]:
Df_female.shape

(1017224, 63)

In [23]:
yf = Df_female['Cantril_ladder']

In [24]:
Xf = Df_female.drop(columns=['Cantril_ladder'])

In [25]:
Xf_train, Xf_test, yf_train, yf_test = train_test_split(Xf, yf, test_size=0.1, random_state=42)

## Model and Tuning Hyper

### Male Model

In [26]:
model = xgb.XGBRegressor(objective='reg:squarederror', device = 'cuda', tree_method='gpu_hist', 
                         n_estimators=500, learning_rate=0.01, max_depth=5, 
                         random_state=42, enable_categorical=True  )
model.fit(Xm_train, ym_train)

[13:15:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "device" } are not used.



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [27]:
ym_pred = model.predict(Xm_test)

In [28]:
r2_score(ym_test, ym_pred) * 100

35.345814761344485

In [29]:
ym_train_pred = model.predict(Xm_train)

In [30]:
r2_score(ym_train, ym_train_pred) * 100

36.214464768212785

In [31]:
n_estimators_list = list(range(100, 1_100, 100))
learning_rate_list = [0.001, 0.01, 0.1]
max_depth_list = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
subsample_list = [0.6, 0.7, 0.8, 0.9, 1]

In [ ]:
results_list = []
for n_estimators in n_estimators_list:
    for learning_rate in learning_rate_list:
        for max_depth in max_depth_list:
            for subsample in subsample_list:
                model = xgb.XGBRegressor(objective='reg:squarederror', 
                                         tree_method='gpu_hist',  ## on linux: device = 'cuda', tree_method='hist', 
                                         n_estimators = n_estimators, 
                                         learning_rate = learning_rate, 
                                         max_depth = max_depth, 
                                         subsample = subsample,
                                         random_state=42, enable_categorical=True  )
                model.fit(Xm_train, ym_train)
                ym_pred = model.predict(Xm_test)
                test_r2 = r2_score(ym_test, ym_pred) * 100
                ym_train_pred = model.predict(Xm_train)
                train_r2 = r2_score(ym_train, ym_train_pred) * 100
                row = [n_estimators, learning_rate, max_depth, 
                       subsample, train_r2, test_r2]
                print(row)
                model = None
                ym_pred = None
                ym_train_pred = None
                results_list.append(row)

[100, 0.001, 3, 0.6, -346.9465146057738, -345.6614864532208]
[100, 0.001, 3, 0.7, -346.94511815918577, -345.6588582299794]
[100, 0.001, 3, 0.8, -346.945979315638, -345.66037200174094]
[100, 0.001, 3, 0.9, -346.9450669262052, -345.65906585005706]
[100, 0.001, 3, 1, -346.9437464794587, -345.65778195979783]
[100, 0.001, 4, 0.6, -346.46283379904037, -345.1830119581036]
[100, 0.001, 4, 0.7, -346.45976203634837, -345.18123516374874]
[100, 0.001, 4, 0.8, -346.46218569776295, -345.18408347586444]
[100, 0.001, 4, 0.9, -346.4604923609569, -345.1822236620651]
[100, 0.001, 4, 1, -346.4580370637822, -345.1797350393475]
[100, 0.001, 5, 0.6, -346.128897802993, -344.84157661217193]
[100, 0.001, 5, 0.7, -346.127099842922, -344.83896621101104]
[100, 0.001, 5, 0.8, -346.1288610316, -344.84036834428633]
[100, 0.001, 5, 0.9, -346.12730377965005, -344.8380758381987]
[100, 0.001, 5, 1, -346.12494584822446, -344.834050753722]
[100, 0.001, 6, 0.6, -345.89156683411034, -344.59520775945026]
[100, 0.001, 6, 0.7, 